## EEG signal classification
### Introduction
In this assignment, we use simple binary classifiers (e.g., NCC and LDA) to accomplish a task in the field of Brain Computer Interface (BCI) called P300 speller paradigm. Our goal is to predict the cognitive state corresponding to the character (target vs. non-target) on the speller where the subject was paying attention through P300 (a kind of deflection in EEG caused by certain stimuli).

### Data description
Electroencephalography (EEG) data was recorded during a copy-spelling BCI Experiment at the Berlin BCI group (http://www.bbci.de). The data set consists preprocessed EEG data $X\in \mathbb{R}^{5\times62\times 5322}$ and stimulus labels $Y\in \mathbb{R}^{2\times 5322}$ during a copy-spelling paradigm with a P300 speller. The data matrix $X$ contains 5 selected time windows of EEG activity at 62 electrodes after a visual stimulus was presented on the screen in front of the subject. If the first row of $Y$ is 1, the stimulus was a target stimulus, if the second row of $Y$ is 1, the stimulus was a non-target stimulus. Hence, in total, we have 5322 EEG data samples where each is associated with a label y (target or non-target).

The classifiers are trained on the 5·62 = 310 concatenated (across time and electrodes) features to predict target stimuli from the EEG data. We further convert stimulus labels $Y$ into a single vector whose elements take values on 1 or -1.

In [68]:
import pylab as pl
import scipy as sp
import numpy as np
from scipy.linalg import eig
from scipy.io import loadmat
import pdb

In [69]:
def load_data(fname):
    # load the data
    data = loadmat(fname)
    X,Y = data['X'],data['Y']
    # collapse the time-electrode dimensions
    X = sp.reshape(X,(X.shape[0]*X.shape[1],X.shape[2]))
    # transform the labels to (-1,1)
    Y = sp.sign((Y[0,:]>0) -.5)
    return X,Y

In [70]:
X,Y = load_data('bcidata.mat')
print(X.shape)
print(Y.shape)

(310, 5322)
(5322,)


<ipython-input-69-c780e27ddfc9>:6: DeprecationWarning: scipy.reshape is deprecated and will be removed in SciPy 2.0.0, use numpy.reshape instead
  X = sp.reshape(X,(X.shape[0]*X.shape[1],X.shape[2]))
<ipython-input-69-c780e27ddfc9>:8: DeprecationWarning: scipy.sign is deprecated and will be removed in SciPy 2.0.0, use numpy.sign instead
  Y = sp.sign((Y[0,:]>0) -.5)


### Task 1: training binary classifiers
How well can we predict the cognitive state from the EEG data?

**Task 1.1**: Implement a nearest centroid classifier which predicts +1 if $|\!| \mathbf{\mu_+} - \mathbf{x}|\!|_2 < |\!| \mathbf{\mu_-} - \mathbf{x}|\!|_2$, where $\mathbf{\mu_{+/-}}$ are means of the positive (respectively negative) class, that is $\mathbf{\mu_+} = \frac{1}{N_+}\sum_i^{N_+}\mathbf{x}_{+i}$ and $\mathbf{x}_{+i}$ is the $i$th data point of the $+$ class.

In [77]:
def train_ncc(X,Y):
    """ Train a nearest centroid classifier
    Args:
        X: numpy array of shape (310, 5322)
        Y: numpy array of shape (5322,)
        
    Returns:
        w (weight vector): numpy array of shape (310,)
        b (bias term): scalar 
    """
    #####################################
    #       START YOUR CODE HERE        #
    #####################################
    x1, x2 = X.shape
    X_minus = np.zeros(1)
    X_minus.resize(x1, 0)
    X_plus = np.zeros(1)
    X_plus.resize(x1, 0)
    
    for i in range(len(Y)):
        if Y[i] < 0:
            X_minus = np.c_[X_minus, X[:,i]]
        else:
            pass
        return X_minus
    
    for n in range(len(Y)):
        if Y[n] > 0:
            X_plus = np.c_[X_plus, X[:, i]]
        else:
            pass
        return X_plus
    
    n_plus = len(np.transpose(X_plus))
    n_minus = len(np.transpose(Y_minus))
    m_plus = np.zeros(1)
    m_plus.resize(x1, 0)
    m_minus = np.zeros(1)
    m_minus.resize(x1, 0)
    
    for m in range(n_plus):
        m_plus += X_plus[:,m]
        return m_plus

    m_plus = m_plus/n_plus

    for s in range(n_minus):
        m_minus += X_minus[:,s]
        return m_minus

    m_minus = m_minus/n_minus

    w = m_plus - m_minus
    
    b = -0.5 * (np.linalg.norm(m_plus)-np.linalg.norm(m_minus))
    
    ######################################
    #        END OF YOUR CODE            #
    ######################################
    return w, b


array([[-1.73408991e+01],
       [-1.74764995e+01],
       [-1.48772493e+01],
       [-1.14791168e+01],
       [-1.20370157e+01],
       [-1.55197727e+01],
       [-1.07775567e+01],
       [-1.06137086e+01],
       [-1.01113254e+01],
       [-9.26754801e+00],
       [-8.76605379e+00],
       [-7.20453583e+00],
       [-7.52475676e+00],
       [-7.06070178e+00],
       [ 8.09870725e-02],
       [-7.40422299e+00],
       [-5.90659273e+00],
       [-5.80537359e+00],
       [-4.26514719e+00],
       [-3.31375731e+00],
       [-2.61299388e+00],
       [-3.14440642e+00],
       [-4.82042406e+00],
       [-4.66999407e+00],
       [-2.86271707e+00],
       [-2.82960540e+00],
       [-8.06546935e+00],
       [-4.41263532e+00],
       [-2.89383837e+00],
       [-1.98348486e+00],
       [-1.53756761e+00],
       [-1.89955484e+00],
       [-2.07636518e+00],
       [ 4.83853484e-01],
       [-1.87917050e+00],
       [-4.06357089e+00],
       [-4.46823364e+00],
       [-1.83960103e+00],
       [-2.0

**Task 1.2**: Implement a linear discriminant analysis (LDA) classifier; optimize a vector $\mathbf{w}\in \mathbb{R}^{310}$ such that 
$$\text{argmax}_{\mathbf{w}}\frac{\mathbf{w}^TS_B\mathbf{w}}{\mathbf{w}^TS_W\mathbf{w}}$$
where
$$S_B = (\mathbf{\mu_+} - \mathbf{\mu_-})(\mathbf{\mu_+} - \mathbf{\mu_-})^T$$
$$S_W = \frac{1}{N_+}\sum_i(\mathbf{x_{+i}} - \mathbf{\mu_+})(\mathbf{x_{+i}} - \mathbf{\mu_+})^T + \frac{1}{N_-}\sum_j(\mathbf{x_{-j}} - \mathbf{\mu_-})(\mathbf{x_{-j}} - \mathbf{\mu_-})^T$$

In [89]:
def train_lda(X,Y):
     """ Train a LDA classifier
    Args:
        X: numpy array of shape (310, 5322)
        Y: numpy array of shape (5322,)
        
    Returns:
        w (weight vector): numpy array of shape (310,)
        b (bias term): scalar 
    """
    #####################################
    #       START YOUR CODE HERE        #
    #####################################
    
    
    X_plus = np.zeros(1)
    x3, x4 = X.shape
    X_plus.resize(x3, 0)
    X_minus = np.zeros(1)
    X_minus.resize(x3, 0)
    
    for i in range(len(Y)):
        if Y[i] > 0:
            X_plus = np.c_[X_plus, X[:,i]]
        else:
            pass
        return X_plus
    
    for n in range(len(Y)):
        if Y[i] < 0:
            X_minus = np.c_[X_minus, X[:,n]]
        else:
            pass
        return X_minus
    
    m_plus = np.zeros(1)
    m_plus.resize(x3, 0)
    m_minus = np.zeros(1)
    m_minus.resize(x3, 0)
    n_plus = len(np.transpose(X_plus))
    n_minus = len(np.transpose(X_minus))
    
    for m in range(n_plus):
        m_plus += X_plus[:,m]
        return m_plus
    
    m_plus = m_plus/n_plus
    
    for s in range(n_minus):
        m_minus += X_minus[:,s]
        return m_minus
    
    m_minus = m_minus/n_minus
    
    Sb = np.dot(m_plus-m_minus, m_plus-m_minus)
    
    xm_plus = np.zeros(1)
    xm_plus.resize(x3, 0)
    xm_minus = np.zeros(1)
    xm_minus.resize(x3, 0)
    
    for q in range(n_plus):
        xm_plus += np.dot((X_plus[:,q]-m_plus),(X_plus[:,q]-m_plus))
        return xm_plus
    xm_plus = xm_plus/n_plus
    
    for r in range(n_minus):
        xm_minus += np.dot((X_minus[:,r]-m_minus),(X_minus[:,r]-m_minus))
        return xm_minus
    xm_minus = xm_minus/n_minus
    
    Sw = xm_plus + xm_minus
    
    w = eig(Sb, Sw)
    
    b = 0.5 * (np.dot(w, m_plus)+np.dot(w, m_minus))
    
    ######################################
    #        END OF YOUR CODE            #
    ######################################
    return w,b


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 16)

Note that, we train both classifiers on 70% of the data and test it on 30% of the data. We select the data points for training and test randomly without overlaping.

The function `compare_classifiers()` below has been implemeted to compare the performance of two classifiers

In [60]:
def compare_classifiers():
    '''
    compares nearest centroid classifier and linear discriminant analysis
    '''
    fname = 'bcidata.mat'
    X,Y = load_data(fname)

    permidx = sp.random.permutation(sp.arange(X.shape[-1]))
    trainpercent = 70.
    stopat = int(sp.floor(Y.shape[-1]*trainpercent/100.))
    #pdb.set_trace()
    
    X,Y,Xtest,Ytest = X[:,permidx[:stopat]],Y[permidx[:stopat]],X[:,permidx[stopat:]],Y[permidx[stopat:]]

    w_ncc,b_ncc = train_ncc(X,Y)
    w_lda,b_lda = train_lda(X,Y)
    fig = pl.figure(figsize=(12,5))

    ax1 = fig.add_subplot(1,2,1)
    #pl.hold(True)
    ax1.hist(w_ncc.dot(Xtest[:,Ytest<0]))
    ax1.hist(w_ncc.dot(Xtest[:,Ytest>0]))
    ax1.set_xlabel('$w^{T}_{NCC}X$')
    ax1.legend(('non-target','target'))
    ax1.set_title("NCC Acc " + str(sp.sum(sp.sign(w_ncc.dot(Xtest)-b_ncc)==Ytest)*100/Xtest.shape[-1]) + "%")
    ax2 = fig.add_subplot(1,2,2)
    ax2.hist(w_lda.dot(Xtest[:,Ytest<0]))
    ax2.hist(w_lda.dot(Xtest[:,Ytest>0]))
    ax2.set_xlabel('$w^{T}_{LDA}X$')
    ax2.legend(('non-target','target'))
    ax2.set_title("LDA Acc " + str(sp.sum(sp.sign(w_lda.dot(Xtest)-b_lda)==Ytest)*100/Xtest.shape[-1]) + "%")
    pl.savefig('ncc-lda-comparison.pdf')


### Task 2 
Run the function compare_classifiers(). Point out or discuss any interesting observations that you may have about the obtained results.

**Your answer here:...**

In [61]:
compare_classifiers()

<ipython-input-5-c780e27ddfc9>:6: DeprecationWarning: scipy.reshape is deprecated and will be removed in SciPy 2.0.0, use numpy.reshape instead
  X = sp.reshape(X,(X.shape[0]*X.shape[1],X.shape[2]))
<ipython-input-5-c780e27ddfc9>:8: DeprecationWarning: scipy.sign is deprecated and will be removed in SciPy 2.0.0, use numpy.sign instead
  Y = sp.sign((Y[0,:]>0) -.5)
<ipython-input-60-185415d21f8a>:8: DeprecationWarning: scipy.arange is deprecated and will be removed in SciPy 2.0.0, use numpy.arange instead
  permidx = sp.random.permutation(sp.arange(X.shape[-1]))
<ipython-input-60-185415d21f8a>:10: DeprecationWarning: scipy.floor is deprecated and will be removed in SciPy 2.0.0, use numpy.floor instead
  stopat = int(sp.floor(Y.shape[-1]*trainpercent/100.))


ValueError: too many values to unpack (expected 2)

### Task 3: N-fold Cross-validation

Estimate the **generalization performance** of each classifier. Implement the function `crossvalidate` below. Test each classifier on the training set data and on the test data data. Compare the prediction accuracies.

In [ ]:
def crossvalidate(X,Y,f=10,trainfunction=train_lda):
    """ Test generalization performance of a linear classifier
    Args:
        X: numpy array of shape (310, 5322)
        Y: numpy array of shape (5322,)
        
    Returns:
        acc_train: numpy array of shape (10,) to store the train accuracies of 10 folds
        acc_test: numpy array of shape (10,) to store the test accuracies of 10 folds
    """

    #####################################
    #       START YOUR CODE HERE        #
    #####################################
    pass
    ######################################
    #        END OF YOUR CODE            #
    ######################################

    return acc_train,acc_test


In [ ]:
X,Y = load_data('bcidata.mat')
crossvalidate(X,Y,f=10,trainfunction=train_lda)

In [ ]:
X,Y = load_data('bcidata.mat')
crossvalidate(X,Y,f=10,trainfunction=train_ncc)